<a href="https://colab.research.google.com/github/JakeOh/202011_itw_bd21/blob/main/lab_ml/ml21_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   IMDB 데이터 셋을 다운로드 - 가장 자주 사용되는 500개의 어휘 사전을 사용.
*   각 샘플(리뷰)의 최대 단어 숫자가 100이 되도록 자르거나 패딩.
*   LSTM 순환 층을 2개를 연결하는 모델을 생성
    *   Input --> Embedding --> LSTM  --> LSTM --> Output
        *   LSTM 층의 unit 개수 = 16
        *   LSTM 층의 dropout 비율 = 0.3
*   모델 컴파일
*   모델 훈련
*   학습 곡선 - 손실 곡선, 정확도 곡선
*   모델 평가